In [52]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from scipy.stats import chisquare

# Lecture 8


In this lecture, we are going to compare algorithms that aim to split a dataset into disjoint subsets. Given a collection of points $D = \{ x_1,\ldots,x_N \}$ we would like to find a function $f\colon D\to \{1,\ldots,k\}$. We are going to do this two different way: 

1. Supervised: 
    1. K-NN
2. Unsupervised: 
    1. K-Means, 
    2. DBScan, 
    3. Hiearchical Clustering.

The task is called *clustering* if the algorithm is unsupervised, and is called *classification* if the algorithm is supervised.

## The Dataset

We are going to use a hyperspectral image dataset called [Indian Pine](https://www.kaggle.com/datasets/abhijeetgo/indian-pines-hyperspectral-dataset).

The data is an image of size $145\times 145$ where each pixel consists of 220 bands. Also, the pixels are labeled by numbers between $0$ to $16$ where $0$ is unclassified while other 16 correspond to certain crop types. 



In [28]:
pine = loadmat('./data/Indian_pines.mat')['indian_pines']
pine_gt = loadmat('./data/Indian_pines_gt.mat')['indian_pines_gt']

In [30]:
X = pine.reshape((145*145,220))
y = pine_gt.reshape(145*145)

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)

In [77]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=uint8)

## K-Nearest Neighbors (K-NN)

K-NN is a supervised algorithm which means our data set consists of pairs $(x^{(i)},y^{(i)})$ where $x^{(i)}\in \mathbb{R}^n$ and $y^{(i)}$ belongs to a finite set of labels. We are also given an odd number $k$.  If we are given a new point $x\in\mathbb{R}^n$ here is how we define $f(x)$:

1. Find $k$-closest points $x^{(1)},\ldots,x^{(k)}$ from the training set $D$ whose labels $y^{(1)},\ldots,y^{(k)}$ are known.
2. Find the majority label among $y^{(1)},\ldots,y^{(k)}$ and return as $f(x)$.

### Example

In [89]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train,y_train)
predicted = model.predict(X_test)
cm = confusion_matrix(y_test,predicted)

In [90]:
model.score(X_test,y_test)

0.728552406315389

In [91]:
chisquare(cm,axis=None)

Power_divergenceResult(statistic=288758.14228647517, pvalue=0.0)

## K-Means

K-Means is a unsupervised algorithm. We have data points $D = \{ x^{(1)},\ldots,x^{(N)} \}$ and a number $k$. Here is a short description of the algorithm:

1. Start with randomly chosen points $c^{(1)},\ldots,c^{(k)}$ from $D$ and $k$ empty sets $S^{(1)},\ldots,S^{(k)}$.
2. For each point $x^{(i)}$ in $D$, find the closest centroid $c^{(j)}$ and add $x^{(i)}$ to $S^{(j)}$.
3. Find the center of each $S^{(i)}$ and call them $d^{(1)},\ldots,d^{(k)}$.
4. If each $c^{(i)}$ and $d^{(i)}$ are close enough terminate the algorithm, if not assign $c^{(i)}\leftarrow d^{(i)}$ and go to step 2.

### Example


In [82]:
k = len(np.unique(y))
model = KMeans(n_clusters=k)
predicted = model.fit_predict(X)
cm = confusion_matrix(y,predicted)
chisquare(cm,axis=None)

Power_divergenceResult(statistic=165879.0181212842, pvalue=0.0)

## Hierarchical Clustering

Hierarchical clustering algorithm is an unsupervised algorithm. The algorithm gradually merge points into clusters. It does this by placing $B_\epsilon(x)$ on each $x\in D$. We merge two points $x_1$ and $x_2$ at a specific $\epsilon$ if $B_\epsilon(x_1)$ and $B_\epsilon(x_2)$ intersect. While this is straightforward for individual pairs of points, one has to decide how clusters of points merge when $\epsilon$ is large enough. This is called the *linkage* method. 

Let us use $ d_{(ij)k} $ to denote the distance between the clusters $ C_{k} $ and $ C_{ij} = C_i \cup C_j $ which is merged in a single cluster.  One calculates the distance 
\\[ d_{(ij)k} = \alpha_{ijk} d_{ik} +\alpha_{jik} d_{jk}+ \beta_{ijk} d_{ij} + \gamma|d_{ik}-d_{jk}| \\] 
for parameters $\alpha_{ijk}$, $\beta_{ijk}$ and $\gamma$ to be determined.  

The most frequently used linkage methods are 

1. Single
2. Complete
3. Average
4. Ward

The parameters for commonly used methods of calculating distances between clusters.

<table>
    <tr><th style="width:100px">Linkage</th>
        <th style="width:100px">$\alpha_{ijk}$</th>
        <th style="width:100px">$\beta_{ijk}$</th>
        <th style="width:100px">$\gamma$</th></tr>
    <tr><td>Single</td>
        <td>$\frac{1}{2}$</td>
        <td>0</td>
        <td>$-\frac{1}{2}$</td></tr>
    <tr><td>Complete</td>
        <td>$\frac{1}{2}$</td>
        <td>0</td>
        <td>$\frac{1}{2}$</td></tr>
    <tr><td>Average</td>
        <td>$\frac{n_i}{n_i+n_j}$</td>
        <td>0</td>
        <td>0</td></tr>
    <tr><td>Ward</td>
        <td>$\frac{n_i+n_k}{n_i+n_j+n_k}$</td>
        <td>$\frac{-n_k}{n_i+n_j+n_k}$</td>
        <td>0</td></tr>
</table>

### Example

In [95]:
k = len(np.unique(y))
model = AgglomerativeClustering(n_clusters=k, 
                                linkage="complete", 
                                affinity='manhattan')
predicted = model.fit_predict(X)
chisquare(confusion_matrix(y,predicted),axis=None)

Power_divergenceResult(statistic=273278.3458739595, pvalue=0.0)

## DB-Scan

DB-Scan is a unsupervised algorithm. So, we only have data points $x^{(1)},\ldots,x^{(N)}$ and no output labels. The parameters are 

1. A real number $\epsilon>0$, and 
2. A natural number $N$

The key concepts for this algorithm are

1. Core point
2. Density connected points
3. Density reachable points
   1. Direct
   2. Indirect
4. Border points
5. Outliers

### Core points

A point $x\in D$ is called a *core point* if $B_\epsilon(x)$ the ball with radius $\epsilon$ centered at $x$ contains $N$ points from $D$.

### Density connected points

A pair of points $x$ and $y$ are called *density connected* if there is a core point $w$ such that $x,y\in B_\epsilon(w)$.

### Density reachable points

A point $x\in D$ is called a *directly density reachable* if $x$ is lies inside $B_\epsilon(u)$ the disk of radius $\epsilon$ centered at a core point $u$. 

We call a point *indirectly density reachable* if there is a link of density connected points $x^{(1)},\ldots,x^{(p)}$ i.e. $d(x^{(i)},x^{(i+1)})<\epsilon$ for $i=1,\ldots,p-1$ and $d(x^{(p)},x)<\epsilon$.

### Border points

A point $x$ is called a *border point* if it is a density reachable point but $B_\epsilon(x)$ has less than $N$ points from $D$.

### Outlier points

A point $x$ is called an outlier if $B_\epsilon(x)$ contains less than $N$ points and $x$ is not a density reachable point.

### The Algorithm

We put point a collection of points $x^{(1)},\ldots,x^{(\ell)}$ in the same cluster if they are density reachable from each other.

### Example

In [98]:
model = DBSCAN()
model.fit(X)
predicted = model.fit_predict(X)
chisquare(confusion_matrix(y,predicted),axis=None)

Power_divergenceResult(statistic=1967321.854696789, pvalue=0.0)